# Implementation of 2 - sided CVaR variants

This notebook shows the implementation of the 2-sided CVaR Economic Dispatch Model discussed in [CVaR ED primer notebook](https://github.com/jdlara-berkeley/Julia_testcodes/blob/master/dispatch_models/CvAR%20ED%20primer.ipynb). It is implemented using the framework of power systems modeling in Julia developed and discussed in this [repository](https://github.com/jdlara-berkeley/Julia_testcodes). 

The example case is the [Small Test Systems for Power System Economic Studies](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=5589973), modified to include 2 PV-Plants for which there is a probabilistic forecast available. The process includes the modification of the ED problem into the Security Constrained Economic Dispatch (SCED) problem. 

The final result is an implementation of the two sided CVaR in the SCED. 

In [1]:
using Distributions
using JuMP
using Plots
using Gurobi
using LaTeXStrings 
using Ipopt
gr();

In [2]:
d1 = Normal(58,15)
d2 = Normal(40,12)
solar_output1 = 17:5:95 #In MW
solar_output2 = 7:5:85 #In MW
p1 = pdf(d1,solar_output1)
p2 = pdf(d2,solar_output2) 
#Julia doesn't have a meshgrid command, so all the combinations of the solar output have to 
# arranged manually. 
solar_output = [0;0];
p = []
for (i, val_i) in enumerate(solar_output1)
    for (j, val_j) in enumerate(solar_output2)
        
        solar_output = hcat(solar_output,[val_i;val_j])
        push!(p,p1[i]*p2[j]) #Joint probabilities, assumming no particular correlation. This has to be captured by
                             #the probabilistic forecast.        
    end
end
solar_output = solar_output[:,2:end]';
p_n_pv = p/sum(p) #Normalized probability space
M = length(p_n_pv);
#The system data for the rest of the components is as follows
load = 175;
generator_max = 85;
generator_min = 50;

In [3]:
println([mean(Truncated(d1, 17, 95))],[mean(Truncated(d2, 7, 85))])
scatter(solar_output[:,1],solar_output[:,2],leg=false, zcolor=p, xlabel = "PV sys 1 [MW]", ylabel = "PV sys 2 [MW]",
        xticks = 2:10:95, yticks = 2:10:95, cbar=true, markerstrokealpha = 0.1)
scatter!([mean(Truncated(d1, 17, 95))],[mean(Truncated(d2, 7, 85))], annotations=(58,44.5,text("Expected Value",:left)),
    c = :red, m = (8, :diamond), xlims = [2,95], ylims = [2,95])

[57.8557][40.1052]


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 12 
 
 
 22 
 
 
 32 
 
 
 42 
 
 
 52 
 
 
 62 
 
 
 72 
 
 
 82 
 
 
 92 
 
 
 2 
 
 
 12 
 
 
 22 
 
 
 32 
 
 
 42 
 
 
 52 
 
 
 62 
 
 
 72 
 
 
 82 
 
 
 92 
 
 
 PV sys 1 [MW] 
 
 
 PV sys 2 [MW] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<image width="18" height="353" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAABIAAAFhCAYAAAB9D/qMAAACCUlEQVR4nO2cwY0DMQwD5WD7r+K6
TGJdCcpjHgNCKoAwSVH2Glifz/3rAupFgFRVPbc/DFBDQBy1vjJqRo0wapDYRmoUUOk0olb01Gr0
A9AbAQKp2VwT2n/6ywDVhYAOBERGhNIIEhu0H6O29o9A514GiFqRUiMfNcg1XdaU9jNiC+3ndpFc
jaInJEYt1v66yPVRPdWr0Qy0Gg2lpIbgcEDkhSYEFKwRR41JCGr/YYBA+5kVkZe+mEY2atSKsjVi
sJI14sZIM4sSUgNHrU4jW2c/vRpNRUbENiGprJH2QxqBEfFptOkfgWwnth0jcyn7KDciNiDhYfT6
ztm20Brtt3W2UKObS03nWrJG20czUPAuEmy/jlq0RpVLjcsagpM8RkD7oYZM1oijhsDshPylhNSo
wSbUaCMyVjQ1Lv1MGXcRCkgntrCPhGdI6qCF7SLU0Y/cRSCg6PRTQMH2MzjKPrJppKQmG7XRY0So
0do/AUXbDwGBOy1TxlG79o9ADf3nQX5mQUA6sYP7yJh+qLOxFZGXdRA14/CXUTNGBLPfljWjRjpq
XPqprAkPETb7hX3U0H+wQmr3YNRkQEb7DxYRm0aY/ZTYJDXotSFQI9uTPNHpX/uHAick9HKJcIx0
MDUsIsEacdQgsakVCceIkZrMNWVEKI2oPiroux98tyY3Ihw1zn6bRhw1COgfmrjn0e4vLFYAAAAA
SUVORK5CYII=
" transform="translate(544, 8)"/>
 
 
 0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 
 
 Expected Value

The formulation being tested is as follows:

$$
\begin{aligned}
\min_{P_g, \delta_1, \delta_2} \ &-(\delta_1 + \delta_2) + (\delta_1 - \delta_2)^2 + P_g\\
       s.t.\ & Pg \in \mathcal{X}_g\\
             & - \left(P_g +\sum_{k=1}^M u_1^k \sum_{n=1}^N P^k_{pv,n} - P_L \right) \ge \delta_1\\
             & \sum_{k=1}^M u_1^k = 1\\
             & 0 \le u_1^k \le \frac{1}{\varepsilon}p^k &\forall k = 1,...,M\\
             & P_g +\sum_{k=1}^M u_2^k \sum_{n=1}^N P^k_{pv,n} - P_L  \ge \delta_2\\
             & \sum_{k=1}^M u_2^k = 1\\
             & 0 \le u_2^k \le \frac{1}{\varepsilon}p^k &\forall k = 1,...,M\\ 
             & \delta_1 \ge 0, \ \delta_2 \ge 0
\end{aligned}
$$

In [44]:
ϵ = 0.05
cvar_edv3 = Model(solver=IpoptSolver())

k_1 = 50; 
k_2 = 14; 

#Generator feasible space
@variable(cvar_edv3, generator_min <= p_g <= generator_max)

#Slack variables for the power balance
@variable(cvar_edv3, delta_1 >= 0)
@variable(cvar_edv3, delta_2 >= 0)

#Slack Variables CVaR
@variable(cvar_edv3, 0<= u1[k = 1:M] <= p_n_pv[k]/(ϵ));
@variable(cvar_edv3, 0<= u2[k = 1:M] <= p_n_pv[k]/(ϵ));

#Robust Power balance constraints 
@constraint(cvar_edv3,  -((p_g + sum((u1[k]*(sum(solar_output[k,:]) - load) for k = 1:M)))) >= delta_1);
@constraint(cvar_edv3,   (p_g + sum((u2[k]*(sum(solar_output[k,:]) - load) for k = 1:M))) >= delta_2);

@constraint(cvar_edv3, sum(u1[k] for k = 1:M)  == 1);
@constraint(cvar_edv3, sum(u2[k] for k = 1:M)  == 1);

@objective(cvar_edv3, Min, 0.95*(-delta_1-delta_2) + 0.05*((k_1*delta_1 - k_2*delta_2)^2 + 10*p_g))
solve(cvar_edv3) 
                                                 #shed       #Curt    

This is Ipopt version 3.12.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      512
Number of nonzeros in inequality constraint Jacobian.:      516
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:      515
                     variables with only lower bounds:        2
                variables with lower and upper bounds:      513
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        2
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

:Optimal

In [45]:
println(getvalue(delta_1))
println(getvalue(delta_2))
println(getvalue(p_g))

12.797244758560057
45.69087413802281
85.0


In [30]:
k_1 = 100; 
k_2 = 120; 
δ1 = 0:5:100
δ2 = 0:5:100
f(δ1, δ2) = begin 
        -(δ1 + δ2) + (k_1*δ1 - k_2*δ2)^2
    end
g(δ1, δ2) = begin 
        -(k_1*δ1 + k_2*δ2) + (k_1*δ1 - k_2*δ2)^2
    end
X = repmat(δ1',length(δ2),1)
Y = repmat(δ2,1,length(δ1))
Z1 = map(f,X,Y)
Z2 = map(g,X,Y)
surface(Z1,fill=true, leg = false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 131.496,274.086 131.496,274.086 128.133,276.992 131.496,274.086 131.496,258.97 129.815,260.423 131.496,258.97 131.496,243.854 128.133,246.759 131.496,243.854 
 131.496,228.738 129.815,230.19 131.496,228.738 131.496,213.621 128.133,216.527 131.496,213.621 131.496,198.505 129.815,199.958 131.496,198.505 131.496,183.389 
 128.133,186.295 131.496,183.389 131.496,168.273 129.815,169.726 131.496,168.273 131.496,153.157 128.133,156.062 131.496,153.157 
 "/>
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 131.496,274.086 146.524,281.758 144.843,283.211 146.524,281.758 171.57,294.544 168.208,297.45 171.57,294.544 196.617,307.331 194.936,308.784 196.617,307.331 
 221.663,320.117 218.301,323.023 221.663,320.117 246.71,332.904 245.029,334.357 246.71,332.904 271.756,345.69 268.394,348.596 271.756,345.69 296.803,358.477 
 295.122,359.93 296.803,358.477 321.849,371.263 318.487,374.169 321.849,371.263 331.868,376.378 
 "/>
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 331.868,376.378 344.478,365.482 346.482,366.505 344.478,365.482 365.494,347.321 369.502,349.367 365.494,347.321 386.511,329.161 388.515,330.184 386.511,329.161 
 407.527,311.001 411.535,313.047 407.527,311.001 428.544,292.84 430.548,293.863 428.544,292.84 449.56,274.68 453.568,276.726 449.56,274.68 470.577,256.52 
 472.581,257.543 470.577,256.52 491.593,238.359 495.601,240.405 491.593,238.359 500,231.095 
 "/>

In [7]:
surface(Z2,fill=true, leg = false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 131.496,274.086 131.496,274.086 128.133,276.992 131.496,274.086 131.496,258.97 129.815,260.423 131.496,258.97 131.496,243.854 128.133,246.759 131.496,243.854 
 131.496,228.738 129.815,230.19 131.496,228.738 131.496,213.621 128.133,216.527 131.496,213.621 131.496,198.505 129.815,199.958 131.496,198.505 131.496,183.389 
 128.133,186.295 131.496,183.389 131.496,168.273 129.815,169.726 131.496,168.273 131.496,153.157 128.133,156.062 131.496,153.157 
 "/>
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 131.496,274.086 146.524,281.758 144.843,283.211 146.524,281.758 171.57,294.544 168.208,297.45 171.57,294.544 196.617,307.331 194.936,308.784 196.617,307.331 
 221.663,320.117 218.301,323.023 221.663,320.117 246.71,332.904 245.029,334.357 246.71,332.904 271.756,345.69 268.394,348.596 271.756,345.69 296.803,358.477 
 295.122,359.93 296.803,358.477 321.849,371.263 318.487,374.169 321.849,371.263 331.868,376.378 
 "/>
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 331.868,376.378 344.478,365.482 346.482,366.505 344.478,365.482 365.494,347.321 369.502,349.367 365.494,347.321 386.511,329.161 388.515,330.184 386.511,329.161 
 407.527,311.001 411.535,313.047 407.527,311.001 428.544,292.84 430.548,293.863 428.544,292.84 449.56,274.68 453.568,276.726 449.56,274.68 470.577,256.52 
 472.581,257.543 470.577,256.52 491.593,238.359 495.601,240.405 491.593,238.359 500,231.095 
 "/>